In [1]:
import os
from google.cloud import storage
from google.cloud.storage.bucket import Bucket
from google.cloud.storage.blob import Blob
from datetime import datetime
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

# TODO: paramaterize
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3
BATCH_SIZE = 32
NUM_IMAGES = 510
NUM_NEIGH = 3 # top 3

In [3]:
# ==============================================================================
# Define helper functions
# ==============================================================================

def _upload_blob_gcs(gcs_uri, source_file_name, destination_blob_name):
    """Uploads a file to GCS bucket"""
    client = storage.Client(project=project)
    blob = Blob.from_string(os.path.join(gcs_uri, destination_blob_name))
    blob.bucket._client = client
    blob.upload_from_filename(source_file_name)

def read_and_decode(filename, reshape_dims=[IMG_HEIGHT, IMG_WIDTH]):
    # Read the file
    img = tf.io.read_file(filename)

    # Convert the compressed string to a 3D uint8 tensor.
    img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)

    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    # This makes the img 1 x 224 x 224 x 3 tensor with the data type of float32
    img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

    # Resize the image to the desired size.
    return tf.image.resize(img, reshape_dims)

def create_embeddings_dataset(embedder, img_path):
    dataset_filenames = []
    dataset_embeddings = []

    list_dir = tf.io.gfile.listdir(img_path)

    for file in list_dir:
        img_tensor = read_and_decode(img_path + "/" + file, [IMG_WIDTH, IMG_HEIGHT])
        embeddings = embedder(img_tensor)
        dataset_filenames.append(img_path + "/" + file)
        dataset_embeddings.extend(embeddings)

    dataset_embeddings = tf.convert_to_tensor(dataset_embeddings)

    return dataset_filenames, dataset_embeddings

In [ ]:
# ==============================================================================
# Download pre-trained model
# ==============================================================================
layers = [
  hub.KerasLayer(
      "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", # TODO: paramaterize
      input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
      trainable=False,
      name='mobilenet_embedding'),
  tf.keras.layers.Flatten()
]

model = tf.keras.Sequential(layers, name='pretrained_mobilenet') # TODO: paramaterize

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
MODEL_NAME = f'pipe-mobilenet_v2-{TIMESTAMP}'
print("MODEL_NAME", MODEL_NAME)

In [ ]:
save_path = os.path.join(saved_model_gcs_bucket, MODEL_NAME) # "gs://", 
print("model save_path", save_path)

model.save(save_path)